In [ ]:
import sys
import os

import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

sys.path.append(os.path.abspath('../classes/'))
from feature_factory import FeatureFactory

In [ ]:
# Trainingsdaten & Testdaten laden
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [ ]:
# Beispiel für die Verwendung FeatureFactory
factory = FeatureFactory(train_df, test_df)

factory.add_time_features()
factory.add_rolling_features(window_size=3)
factory.add_ratio_and_diff_features()
factory.add_aFRR_activation_request_ratio()
factory.add_FRCE_LFCInput_difference()
factory.add_participation_state()
factory.add_demand_FRCE_interaction()

factory.train_data.columns

In [ ]:
# Features
# New features beginn with 'day', ...
features = ['Demand', 'correction', 'correctedDemand',
       'FRCE', 'controlBandPos', 'controlBandNeg', 'LFCInput',
       'aFRRactivation', 'aFRRrequest', 'participationCMO', 'participationIN',
       'correctionEcho', 'BandLimitedCorrectedDemand', 'controlArea', 'hour',
       'day', 'weekday', 'month', 'Demand_RollingMean', 'Demand_RollingStd',
       'Demand_CorrectedDemand_Ratio', 'Demand_CorrectedDemand_Diff',
       'aFRR_Activation_Request_Ratio', 'FRCE_LFCInput_Diff',
       'Participation_State', 'Demand_FRCE_Interaction'] 
X_train = factory.train_data[features]
X_test = factory.test_data[features]

In [ ]:
X_train.isna().sum()          

In [ ]:
# Scaler
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

In [ ]:
# Isolation Forest Modell initialisieren und trainieren
model = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
model.fit(X_train_normalized)

In [ ]:
# Anomalien auf Testdaten vorhersagen und anzeigen
test_df['anomaly'] = model.predict(X_test_normalized)
print(test_df[['Datum_Uhrzeit_CET', 'Demand', 'correctedDemand', 'anomaly']].head())

In [ ]:
# Konvertiere Anomalie-Vorhersagen: -1 (Anomalie) wird zu 1 und 1 (normal) wird zu 0
test_df['anomaly'] = test_df['anomaly'].apply(lambda x: 1 if x == -1 else 0)
submission_df = test_df[['id', 'anomaly']]
submission_df.to_csv('submission.csv', index=False)